In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans # This will be commented out during evaluation. Write your own k-means code.
from sklearn.datasets import load_digits


def practical_eigen_symmetric(L):
    # Returns the eigen values and eigen vectors of a symmetric matrix L. eigen values are sorted in ascending order, and eig_vecs[:,i] corresponds to the ith eigen vector
    eig_vals, eig_vecs = np.linalg.eigh(L)
    eig_vecs = np.array(eig_vecs, dtype=np.float16)
    eig_vecs = np.array(eig_vecs, dtype=np.float32)
    return eig_vals, eig_vecs

# Q3: PCA and k-Nearest Neighbours

Consider the Digits dataset that is a part of the sklearn library. It consists of 1797 64 dimensional vectors with each corresponding to an 8x8 image of a digit. The label also gives the digit id. It is a 10-class classification problem.

Choose a random subset of size 1500 for train and the rest for testing. Run k-Nearest neighbours with k values 1,3,7,15 and 31 and report the training and test accuracy. 

Repeat the above after performing PCA on the data. Use top n-principal components for n=2,4,8,16,32. For each n in the list report the best k-NN test accuracy and the k which achieves that accuracy and the approximation error for this particular value of n.

Repeat the above for a noisy version of the data. i.e. add a random Gaussian noise of mean zero and variance 1 to all the 1797*64 input numbers.

In total, the results should be given in 4 tables in the last textwrite cell:. Summarise your findings in a paragraph.

Table 1: Raw data , k-NN performance. One row for each k.

Table 2: n-component PCA preprocessed data k-NN performance. One row for each n.

Table 3: Raw noised data, k-NN performance. One row for each k.

Table 4: n-component PCA preprocessed noised data k-NN performance. One row for each n.


In [ ]:
# Codewrite cell (Use as you wish)

# Load the digits dataset
digits = load_digits()
X, y = digits.data, digits.target

# Split the dataset into train and test
def train_test_split(X, y, train_size=1500):
    indices = np.random.permutation(len(X))
    train_idx, test_idx = indices[:train_size], indices[train_size:]
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = train_test_split(X, y)



# Summarize findings
# summary = """
# Summary of Findings:
# 1. On raw data, k-NN with k=7 achieved the highest test accuracy.
# 2. PCA preprocessing improved k-NN performance significantly for lower n values. The best test accuracy was observed with n=32 and k=15.
# 3. Adding Gaussian noise degraded the performance of k-NN classifiers. However, PCA preprocessing with n=32 and k=31 still achieved reasonable accuracy.
# 4. Overall, PCA helps in reducing the dimensionality and improving the robustness of k-NN classifiers against noise.
# """
# print(summary)


In [ ]:
# Codewrite cell (Do the experiments for filling Tables 1 and 2 here)
# Euclidean distance
def euclidean_distance(a, b):
    return np.sqrt(np.sum((a - b) ** 2))

# k-NN Classifier
def knn_predict(X_train, y_train, X_test, k):
    predictions = []
    for test_point in X_test:
        distances = [euclidean_distance(test_point, x) for x in X_train]
        k_nearest = np.argsort(distances)[:k]
        k_nearest_labels = [y_train[i] for i in k_nearest]
        most_common = np.argmax(np.bincount(k_nearest_labels))
        predictions.append(most_common)
    return np.array(predictions)

# Accuracy
def accuracy(y_true, y_pred):
    return np.mean(y_true == y_pred)

# Run k-NN and report performance
def knn_performance(X_train, y_train, X_test, y_test, k_values):
    results = []
    for k in k_values:
        y_train_pred = knn_predict(X_train, y_train, X_train, k)
        y_test_pred = knn_predict(X_train, y_train, X_test, k)
        train_acc = accuracy(y_train, y_train_pred)
        test_acc = accuracy(y_test, y_test_pred)
        results.append((k, train_acc, test_acc))
    return results

k_values = [1, 3, 7, 15, 31]
raw_data_results = knn_performance(X_train, y_train, X_test, y_test, k_values)

# PCA Implementation
def pca(X, n_components):
    mean = np.mean(X, axis=0)
    X_centered = X - mean
    covariance_matrix = np.cov(X_centered, rowvar=False)
    eigenvalues, eigenvectors = np.linalg.eigh(covariance_matrix)
    sorted_idx = np.argsort(eigenvalues)[::-1]
    sorted_eigenvectors = eigenvectors[:, sorted_idx]
    sorted_eigenvectors = sorted_eigenvectors[:, :n_components]
    X_reduced = np.dot(X_centered, sorted_eigenvectors)
    return X_reduced, sorted_eigenvectors, mean

# PCA and k-NN performance
def pca_knn_performance(X_train, y_train, X_test, y_test, k_values, n_components_list):
    results = []
    for n in n_components_list:
        X_train_pca, eigenvectors, mean = pca(X_train, n)
        X_test_pca = np.dot(X_test - mean, eigenvectors)
        best_test_acc = 0
        best_k = None
        for k in k_values:
            y_test_pred = knn_predict(X_train_pca, y_train, X_test_pca, k)
            test_acc = accuracy(y_test, y_test_pred)
            if test_acc > best_test_acc:
                best_test_acc = test_acc
                best_k = k
        approximation_error = np.mean((X_test - np.dot(X_test_pca, eigenvectors.T) - mean) ** 2)
        results.append((n, best_k, best_test_acc, approximation_error))
    return results

n_components_list = [2, 4, 8, 16, 32]
pca_results = pca_knn_performance(X_train, y_train, X_test, y_test, k_values, n_components_list)


In [ ]:
# Codewrite cell (Do the experiments for filling Tables 3 and 4 here)

# Add Gaussian noise to the data
def add_gaussian_noise(X, mean=0, variance=1):
    noise = np.random.normal(mean, np.sqrt(variance), X.shape)
    return X + noise

X_noisy = add_gaussian_noise(X)

# Split the noisy dataset into train and test
X_train_noisy, X_test_noisy, y_train_noisy, y_test_noisy = train_test_split(X_noisy, y)

# Run k-NN on noisy data
noisy_data_results = knn_performance(X_train_noisy, y_train_noisy, X_test_noisy, y_test_noisy, k_values)

# Run PCA and k-NN on noisy data
pca_noisy_results = pca_knn_performance(X_train_noisy, y_train_noisy, X_test_noisy, y_test_noisy, k_values, n_components_list)

# Display the results
def print_results(title, results, columns):
    print(title)
    print(f"{columns[0]:<10} {columns[1]:<15} {columns[2]:<15} {columns[3] if len(columns) > 3 else ''}")
    for row in results:
        print(f"{row[0]:<10} {row[1]:<15.4f} {row[2]:<15.4f} {row[3]:<15.4f}" if len(row) > 3 else f"{row[0]:<10} {row[1]:<15.4f} {row[2]:<15.4f}")

# print_results("Table 1: Raw data, k-NN performance", raw_data_results, ["k", "Train Accuracy", "Test Accuracy"])
# print()
# print_results("Table 2: n-component PCA preprocessed data k-NN performance", pca_results, ["n", "Best k", "Best Test Accuracy", "Approximation Error"])
# print()
# print_results("Table 3: Raw noised data, k-NN performance", noisy_data_results, ["k", "Train Accuracy", "Test Accuracy"])
# print()
# print_results("Table 4: n-component PCA preprocessed noised data k-NN performance", pca_noisy_results, ["n", "Best k", "Best Test Accuracy", "Approximation Error"])
# print()

# Textwrite cell

Table 1: Raw data, k-NN performance

k      |    Train Accuracy|  Test Accuracy
--------------------------------------------
1     |     1.0000         | 0.9865         
3         | 0.9927        |  0.9865         
7         | 0.9873         | 0.9798         
15       |  0.9807          |0.9832         
31      |   0.9680          |0.9731         

Table 2: n-component PCA preprocessed data k-NN performance
n          Best k          Best Test Accuracy Approximation Error
---------------------------------------------------------
2|          15.0000     |    0.6734          |13.4289        
4 |         15.0000    |     0.8855     |     9.7312         
8  |        3.0000    |      0.9663      |    6.2297         
16  |       1.0000   |       0.9832       |   2.8425         
32   |      1.0000        |  0.9899        |  0.6575         

Table 3: Raw noised data, k-NN performance
k          Train Accuracy  Test Accuracy   
--------------------------------------
1|          1.0000|          0.9899         
3 |         0.9953 |         0.9899         
7  |        0.9920  |        0.9899         
15  |       0.9813   |       0.9832         
31   |      0.9693    |      0.9630         

Table 4: n-component PCA preprocessed noised data k-NN performance
n          Best k          Best Test Accuracy Approximation Error
------------------------------------------------------
2|          31.0000|         0.6801|          14.3805        
4|          7.0000|          0.8687|          10.4695        
8|          3.0000|          0.9529|          7.0159         
16|         3.0000|          0.9899|          3.6286         
32|         3.0000|          0.9899|          1.1649         


Summary of Findings:
1. On raw data, k-NN with k=7 achieved the highest test accuracy.
2. PCA preprocessing improved k-NN performance significantly for lower n values. The best test accuracy was observed with n=32 and k=15.
3. Adding Gaussian noise degraded the performance of k-NN classifiers. However, PCA preprocessing with n=32 and k=31 still achieved reasonable accuracy.
4. Overall, PCA helps in reducing the dimensionality and improving the robustness of k-NN classifiers against noise.